TODO:
- Banco de Sinônimos
- Gerar Palavras Chaves também com base nos sinônimos
- PosTag
- Random (3 score > 90%, 4 score entre 85% e 90%, 5 score entre 80% e 95%)
- Lema

Exemplos:

https://medium.com/botsbrasil/qual-a-melhor-linguagem-para-se-criar-um-chatbot-381c9d2b1ced

https://medium.com/swlh/a-chatbot-in-python-using-nltk-938a37a9eacc

https://github.com/gunthercox/ChatterBot/tree/master/examples/django_app

https://github.com/parulnith/Building-a-Simple-Chatbot-in-Python-using-NLTK/blob/master/Chatbot.ipynb


In [1]:
import random
import string
import warnings
import numpy as np
import re

In [2]:
from unicodedata import normalize

In [3]:
import nltk

In [4]:
nltk.download('punkt', quiet=True) # first-time use only
nltk.download('wordnet', quiet=True) # first-time use only
nltk.download('popular', quiet=True)

True

# Conversation

In [5]:
cordialities = [
  'oi', 'oi',
  'ola', 'ola',
  'bom dia', 'bom dia',
  'boa tarde', 'boa tarde',
  'boa noite', 'boa noite',
]

In [6]:
goodbyes = [
  'bye', 'bye',
  'tchau', 'tchau',
  'ate logo', 'ate logo',
  'ate mais ver', 'ate mais ver',
  'bon voyage', 'bon voyage',
  'arrivederci', 'arrivederci',
  'ate mais', 'ate mais',
  'adeus', 'adeus', 
  'hasta la vista baby', 'hasta la vista baby',
]

In [49]:
conversations = [
'Abate Humanitário', 'Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do que em outros tipos de abate. ', 
'Abolicionismo', 'Na causa animal, um movimento filosófico e político que sustenta que nenhum animal deva ser explorado, mesmo que com melhores condições ou menos sofrimento. Assim como aconteceu com o abolicionismo humano, que propôs o fim da escravidão de homens e mulheres, o abolicionismo animal busca a abolição de toda forma de escravidão e exploração animal. ', 
'Agave', 'Um substituto do mel com mesma aparência e textura. Tem um sabor adocicado muito próximo do mel também, mas sua origem é um cacto, o mesmo que dá origem à tequila. ', 
'ALF', 'Sigla em inglês para Animal Liberation Front. Em português, Frente de Liberação Animal (FLA). É uma frente internacional formada por ativistas normalmente mascarados que entram em locais onde há animais sendo explorados para resgatá-los. Além do resgate, normalmente os ativistas da ALF causam prejuízo financeiro ao local para que outros animais não sejam explorados depois. ', 
'Aquafaba', 'É a água do cozimento do grão-de-bico ou de outra leguminosa que, após batida em batedeira, se transforma em uma espécie de clara em neve. O mais comum é que a água do grão-de-bico seja usada no preparo da aquafaba e é possível fazer mousses e até suspiros com ela. Uma das maiores invenções da culinária vegana nos últimos anos. A aquafaba foi criada no fim de 2014 por um francês. ', 
'B12', 'Vitamina muito falada no meio vegano por ser encontrada naturalmente apenas em produtos de origem animal. Ela não é, no entanto, produzida por animais. A vitamina B12 é produzida por bactérias e é encontrada em produtos animais porque eles tornam-se depósitos dessa vitamina por consumir alimentos sem muita higienização. A vitamina B12 é essencial para a prevenção de alguns tipos de anemia e para o bom funcionamento do sistema nervoso. Sua reposição por meio de suplementação é obrigatória para vegetarianos. Muitas pessoas que comem carne também apresentam deficiência dessa vitamina e também devem fazer a suplementação. ', 
'Bem-estarismo', 'Forma de pensar de quem acha que basta que os animais sofram menos na pecuária. Os chamados bem-estaristas pregam que os animais devam ser respeitados dentro da indústria, mas não que eles devam simplesmente deixar de ser explorados. Uma bandeira de quem defende o bem-estarismo é o chamado abate humanitário. ', 
'Bem-estarista', 'Quem pratica o bem-estarismo. ', 
'Biomassa de banana', 'Uma massa obtida por meio do cozimento de bananas verdes. Não tem sabor, serve apenas para enriquecer alimentos e como base para brigadeiros, sopas e pães. ', 
'Carne de Jaca', 'Uma espécie de carne branca vegetal feita com a fruta jaca verde. A fruta, que precisa estar verde, é cozida e depois desfiada e pega o tempero que for dado a ela. Não tem gosto da fruta madura, é impossível reconhecer que é jaca. É usada por sua textura muito parecida com a da carne de frango. ', 
'Carne de Soja', 'PTS é uma sigla que significa Proteína Texturizada de Soja. É a forma mais comum e fácil de se fazer a "carne de soja", que se parece muito com carne moída. A PTS é obtida da soja por meio de um processo industrial chamado extrusão. Muitos nutricionistas não recomendam a ingestão exagerada de PTS e alguns recomendam que ela não seja consumida.  ', 
'Carnismo', 'É a crença de que certas espécies podem ser mortas para consumo. É um tipo de ideologia não consciente ou consciente de quem defende o consumo de carne. O termo foi criado pela Phd Melanie Joy, da universidade de Harvard. ', 
'Carnista', 'Quem pratica, de forma consciente ou inconsciente, o carnismo. Procure por carnismo. ', 
'Churrasco Vegano', 'Um churrasco normalmente feito com espetinhos que, no lugar de carnes e queijos, utiliza apenas vegetais ou embutidos feitos à partir de vegetais. ', 
'Cochonilha', 'Corante vermelho proveniente de insetos e largamente utilizado pela indústria de alimentos e cosméticos. Aparece nas embalagens dos produtos como: Cochonilha, Corante Natural Carmim, Ácido Carmínico, CI 75470, INS 120 ou E120. ', 
'Cruelty Free', 'Termo em inglês que significa "livre de crueldade". Usado principalmente na indústria de cosméticos, o termo é bastante traiçoeiro, já que é usado para definir apenas produtos que não foram testados em animais. Há muitos produtos que têm ingredientes de origem animal em sua composição e que, mesmo assim, têm o selo Cruelty Free. O selo, portanto, não garante produtos realmente livres de crueldade. ', 
'Especismo', 'Atribuição de direitos com base na espécie do indivíduo. Exemplo: Pensar que não há problema em matar um porco para comer, mas achar um absurdo que se mate um cachorro para o mesmo fim. ', 
'Especista', 'Aquele que pratica o especismo. ', 
'Esquizofrenia Moral', 'O modo ilusório e confuso de pensar que alguns animais podem ser mortos, enquanto outros são amigos e membros da família. Algo bem parecido com carnismo (procure carnismo). O termo esquizofrenia moral foi cunhado pelo professor norte-americano Gary Francione. ', 
'Falafel', 'Bolinho frito à base de grão-de-bico e ervas em formato ovalado de origem árabe. Sua receita original é vegana. É simplesmente delicioso. ', 
'Free Range', 'Termo em inglês que define produtos oriundos de animais criados "soltos". Mas "soltos", quase sempre, refere-se a grandes galpões fechados, e não a um campo verde e cheio de flores. É um termo muito usado em embalagens de ovos obtidos de galinhas exploradas fora das terríveis gaiolas de bateria. No fim de sua vida útil para a indústria, porém, os animais explorados para produtos "free range" são abatidos, como acontece com todos os outros animais nessa indústria. ', 
'Gaiolas de bateria', 'São gaiolas usadas para confinamento intensivo de animais na indústria. Muito comum na indústria dos ovos, o uso dessas gaiolas é considerado um dos métodos mais cruéis já criados pelo ser humano. Nessas gaiolas, as galinhas exploradas pela indústria dos ovos passam a vida sem poder abrir as asas em um espaço equivalente ao de uma folha de sulfite A4. O confinamento com gaiolas de bateria é comum também na criação de porcos, com gaiolas preparadas para o tamanho desses animais. Eles não podem sequer virar para trás. ', 
'Grãomelete', 'Espécie de omelete feito sem o uso de ovos ou qualquer outro ingrediente de origem animal. Normalmente, é uma massa à base de grão-de-bico que é frita no formato de um omelete. ', 
'Guacamole', 'Um preparado salgado à base de abacate de origem mexicana. É naturalmente vegano em sua receita original. ', 
'Homus', 'Pasta de grão-de-bico de origem árabe que pode ser facilmente preparada em casa. É muito nutritiva e saborosa. ', 
'Lactovegetariano', 'Indivíduo que não consome carnes de nenhum tipo, mas que ainda consome laticínios e seus derivados. ', 
'Leite vegetal', 'Qualquer leite que seja feito a partir de vegetais. Os mais comuns são os de soja, amêndoas, amendoim e coco. Basicamente, é só bater o vegetal com o dobro de água e coar, mas é importante buscar receitas específicas. ', 
'Levedo de Cerveja', 'Procure por levedura de cerveja. ', 
'Levedura de Cerveja', 'Ingrediente facilmente encontrado em supermercados que normalmente é utilizado para substituir a levedura nutricional em receitas veganas. Mas a levedura de cerveja tem um gosto mais puxado para o amargo. É preciso ter cuidado antes de incluir em qualquer receita. ', 
'Levedura Nutricional', 'Procure por Nutritional Yeast. ', 
'Libertação Animal', 'O fim de todo tipo de sofrimento e exploração animal. ', 
'Melado', 'Melado ou melado-de-cana é um derivado da cana-de-açúcar usado para substituir o mel em receitas. A receita vegana mais famosa que utiliza este ingrediente é o pão de melado. O melado é riquíssimo em ferro, mais uma vantagem sobre o mel, além de ser livre de crueldade. ', 
'Natureba', 'Termo informal – e por vezes pejorativo – para definir pessoa que só come coisas naturais e é excessivamente preocupada com a saúde. ', 
'Nutritional Yeast', 'É um ingrediente normalmente importado que dá um sabor parecido com o de queijo às preparações e, de quebra, deixa tudo muito mais nutritivo, pois tem dezenas de vitaminas. O principal fabricante chama-se Red Star. É conhecido no Brasil também como levedura nutricional. Não é a mesma coisa que levedura de cerveja. ', 
'Ovolactovegetariano', 'Indivíduo que não consome carnes de nenhum tipo, mas que ainda consome ovos, laticínios e seus derivados. ', 
'Ovovegetariano', 'Indivíduo que não consome carnes de nenhum tipo, mas que ainda consome ovos. ', 
'PTS', 'PTS é uma sigla que significa Proteína Texturizada de Soja. É a forma mais comum e fácil de se fazer a "carne de soja", que se parece muito com carne moída. A PTS é obtida da soja por meio de um processo industrial chamado extrusão. Muitos nutricionistas não recomendam a ingestão exagerada de PTS e alguns recomendam que ela não seja consumida.  ', 
'PVT', 'Procure por PTS. A sigla PVT significa Proteína Vegetal Texturizada, mas o produto é o mesmo que PTS. ', 
'Santuário de animais', 'Normalmente um sítio ou uma fazenda dedicada a abrigar animais e que não tem visitação pública. Alguns santuários permitem apenas visitação guiada de pequenos grupos, para não estressar ou prejudicar os animais. ', 
'Seitan', 'Também chamado de "carne de glúten", é um alimento à base da proteína do trigo que é muito versátil e pode ser frito, grelhado e preparado de várias formas, onde as pessoas costumam usar carne. ', 
'Senciência', 'É a capacidade de ter sentimentos e sensações. Os seres sencientes, ao contrário das plantas, apresentam sistema nervoso e podem sentir dor, medo, desespero, alegria etc. Gatos, bois, peixes, cachorros, porcos, pássaros e frangos são exemplos de seres sencientes. ', 
'Seres Sencientes', 'Procure por senciência. ', 
'Tempeh', 'Alimento à base de soja fermentada com sabor bem peculiar. Muito nutritivo e versátil, pode ser grelhado e colocado em sanduíches ou refeições. ', 
'Tofu', 'Alimento feito à base de soja que se parece com um queijo, mas praticamente não tem sabor. É uma base, usada para inúmeras receitas salgadas e também doces. ', 
'Traços', 'Traços são possíveis resquícios que ficam no alimento que é produzido em máquinas ou cozinhas que manipulam outros alimentos. Quando um alimento traz na embalagem a informação de que pode haver ali traços de leite, por exemplo, significa que a empresa usa a mesma máquina para fazer alimentos com e sem leite. Por padrão, as empresas precisam fazer uma higienização entre uma produção e outra, ainda assim, a mensagem precisa constar no rótulo, por lei. ', 
'Tutor', 'Indivíduo que tutora, que cuida, de animais. Substitui o termo "dono", uma vez que os animais não devem ser considerados mercadoria. Exemplo: "Ele é o tutor daquele cãozinho lindo.", "Eu tutoro dois gatinhos desde que eles foram resgatados das ruas.". ', 
'Vegan', 'Procure por vegano. Palavra em inglês que significa vegano. ', 
'Veganismo', 'Filosofia de vida que busca excluir, sempre que praticável, todas as formas de exploração animal. Tanto na alimentação, quanto no entretenimento, no vestuário e em todos os outros comportamentos de consumo. É também uma postura política e social contra a exploração animal. ', 
'Veganizar', 'Termo informal utilizado como verbo para pessoas, pratos ou locais que aderiram ao veganismo. Exemplos: "Fulano veganizou.", "Precisamos veganizar a rede de lanchonetes toda.", "A cidade toda está veganizando aos poucos.", "Ele conseguiu veganizar a receita de bolo preferida da família.". ', 
'Vegano', 'Indivíduo que utiliza o vegetarianismo como dieta, ou seja, que não consome nada de origem animal em sua alimentação. E que também não consome nada de origem animal em cosméticos, vestuário e em todos os aspectos da vida. Além disso, veganos não frequentam ou apoiam eventos que causem sofrimento aos animais como rodeios e atividades similares. ', 
'Vegetarianismo', 'Método alimentar que consiste na utilização apenas de vegetais.', 
'Vegetariano', 'Quem utiliza o vegetarianismo como dieta. É comum, porém, pessoas ovolactovegetarianas, ovovegetarianas ou lactovegetarianas serem chamadas apenas de vegetarianas, o que é um erro na visão de muitos especialistas.', 
'Vegetariano Estrito', 'Individuo que se alimenta apenas de vegetais, mas que não necessariamente evita roupas ou cosméticos feitos a partir de animais.',
'Vivissecção', 'Estudos ou experimentações feitas em animais vivos. É cortar o animal vivo.'
]

In [8]:
questions = [
  '', 
  'funciona',
  'me explica funciona',
  'gostaria informacoes sobre',
  'informacao sobre',
  'informacoes sobre',
  'me explique',
]

# Text

## Functions

In [10]:
def Lower(s):
    return s.lower()

In [11]:
def RemoveURL(s):
  # Código-Fonte utilizado para remoção de hashtags e URLs do corpus
  # Remove as hashtags do corpus pattern = re.compile(r'\#\w+') raw = pattern.sub('', raw)
  # Remove as URLs do corpus
  # pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
  # return pattern.sub('', s)
  return re.sub(
      r"(https?:\/\/(?:www\d?\.|(?!www\d?))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\d?\."
      r"[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\d?\.|(?!www\d?))[a-zA-Z0-9]"
      r"+\.[^\s]{2,}|www\d?\.[a-zA-Z0-9]+\.[^\s]{2,})",
      " __url__ ",
      s,
  )

In [12]:
def RemoveEmail(s):
    return re.sub(
      r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", " __email__ ", s
    )

In [13]:
def RemoverNumbers(s):
  # return re.sub(r'[0123456789]',' ',s)
  return re.sub(r"[\d]+[\d\,\.\-\(\) \/:ºª\%]*", " __num__ ", s)

In [14]:
def RemoveAccents(s):
    return normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")

In [15]:
def RemoveStopWords(s):
    stopwords = set(nltk.corpus.stopwords.words("portuguese"))
    for letter in range(97, 123):
        stopwords.add(chr(letter))
    palavras = [i for i in s.split() if not i in stopwords]
    stopwords = set(nltk.corpus.stopwords.words("english"))
    palavras = " ".join(palavras)
    palavras = [i for i in palavras.split() if not i in stopwords]
    return " ".join(palavras)

In [16]:
def RemovePunctuation(s):
    return re.sub(r'[\\_\+\*-./?!,":;()=#\<\>\'|]', " ", s)

In [17]:
def RemoveSmaller(s, num):
    words = [i for i in s.split() if len(i) > num]
    words = " ".join(words)
    return words

In [18]:
def TrateText(s, stemming=False):
    s = Lower(s)
    s = RemoveURL(s)
    s = RemoveEmail(s)
    s = RemoverNumbers(s)
    s = RemoveAccents(s)
    s = RemoveStopWords(s)
    s = RemovePunctuation(s)
    # s = normalize_text(s)
    # s = clear_text(s)
    if stemming:
        s = Stemming(s)
    s = RemoveSmaller(s, 1)
    return s

In [19]:
TrateText('É 1 Indivíduo que utiliza o vegetarianismo como dieta, ou seja, que não consome nada de origem animal em sua alimentação. E que também não consome nada de origem animal em cosméticos, vestuário e em todos os aspectos da vida. Além disso, veganos não frequentam ou apoiam eventos que causem sofrimento aos animais como rodeios e atividades similares. ')

'num individuo utiliza vegetarianismo dieta seja nao consome nada origem animal alimentacao tambem nao consome nada origem animal cosmeticos vestuario todos aspectos vida alem disso veganos nao frequentam apoiam eventos causem sofrimento animais rodeios atividades similares'

In [20]:
usable = []
for question in questions:
  it = iter(conversations)
  for definition, conversation in zip(it, it):
    usable.append(' '.join([TrateText(definition), question]).strip())
    usable.append(conversation)
    usable.append(' '.join([question, TrateText(definition)]).strip())
    usable.append(conversation)


In [21]:
len(usable)

1512

In [23]:
usable[:10]

['abate humanitario',
 'Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do que em outros tipos de abate. ',
 'abate humanitario',
 'Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do que em outros tipos de abate. ',
 'abolicionismo',
 'Na causa animal, um movimento filosófico e político que sustenta que nenhum animal deva ser explorado, mesmo que com melhores condições ou menos sofrimento. Assim como aconteceu com o abolicionismo humano, que propôs o fim da escravidão de homens e mulheres, o abolicionismo animal busca a abolição de toda forma de escravidão e exploração animal. ',
 'abolicionismo',
 'Na causa animal, um movimento filosófico e político que sustenta que nenhum animal deva ser explorado, mesmo que com melhores condições ou menos sofrimento. Assim como aconteceu com o abolicionismo humano, que propôs o fim da e

# ChatterBot

## Train

In [48]:
# !pip install chatterbot

In [29]:
# !pip install chatterbot_corpus

In [ ]:
# sudo pip3 uninstall chatterbot-corpus
# sudo pip3 uninstall chatterbot
# sudo pip3 install PyYAML==5.1
# sudo pip3 install --ignore-installed PyYAML==3.12
# sudo pip3 install chatterbot
# sudo pip3 install chatterbot-corpus
# sudo pip3 uninstall pandas
# sudo pip3 install pandas
# sudo pip3 install --upgrade pandas

In [30]:
from chatterbot.trainers import ListTrainer

In [31]:
from chatterbot import ChatBot

In [32]:
bot = ChatBot('VISTA-SE')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ivo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/ivo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
trainer = ListTrainer(bot)

In [34]:
trainer.train(usable)

List Trainer: [####################] 100%


In [35]:
trainer.train(cordialities)

List Trainer: [####################] 100%


In [36]:
trainer.train(goodbyes)

List Trainer: [####################] 100%


 # IA

## Cosine Distance

In [37]:
from nltk.cluster.util import cosine_distance

In [38]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return cosine_distance(vector1, vector2)

In [39]:
def answer_cosine_distance(sent):
    last = 0
    converse = ''
    sent = TrateText(sent)
    for conversation in usable:
        x = sentence_similarity(sent, conversation)
        if x > last:
          last = x
          converse = conversation

    return last, converse

In [40]:
answer_cosine_distance('vegano')

(1.0, 'pts')

## LSA

In [41]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from nltk.corpus import stopwords

In [42]:
import pandas as pd

In [43]:
def lsa(sent):
    vectorizer = CountVectorizer(min_df=5, ngram_range=(1, 3), stop_words=stopwords.words("portuguese"))

    usable_trated = [TrateText(x) for x in usable]

    dtm = vectorizer.fit_transform(usable_trated)
    lsa = TruncatedSVD(150, algorithm="arpack")

    dtm = dtm.asfptype()
    dtm_model = lsa.fit(dtm)

    dtm_lsa = dtm_model.transform(dtm)
    dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)

    dtm = vectorizer.transform([TrateText(sent)])
    dtm_lsa_exemplo = dtm_model.transform(dtm)
    dtm_lsa_exemplo = Normalizer(copy=False).fit_transform(dtm_lsa_exemplo)

    similarity = np.asarray(np.asmatrix(dtm_lsa) * np.asmatrix(dtm_lsa_exemplo).T)

    res = (pd.DataFrame(similarity, index=usable_trated, columns=['sent'])
    .sort_values(['sent'], ascending=False)
    .head(1))

    text = res.index[0]
    idx = usable_trated.index(text) 
    score = res.values[0][0]

    return idx, text, score

In [52]:
lsa('me explica o que é o veganismo')

(188, 'veganismo', 0.8142119302868118)

In [53]:
lsa('o que é um ovolactovegetariano?')

(136, 'ovolactovegetariano', 1.0000000000000002)

# Bot

In [54]:
while True:
    message = input('Me:')
    message = TrateText(message)
    reply = bot.get_response(message)
    if reply.confidence > 0.8:
        print('ChatBot ', round(reply.confidence, 2), ':', reply)
    else:
        idx, reply, score = lsa(message)

        reply = '?'
        if idx % 2 != 0:
            idx = idx - 1
        for index, item in enumerate(usable):
            if index == idx:
                reply = bot.get_response(item)
                break
        print('ChatBot ', round(score, 2), ':', reply)

    if message in goodbyes:
        break

Me:oi
ChatBot  1.0 : oi
Me:boa tarde
ChatBot  1.0 : boa tarde
Me:o que e veganismo?
ChatBot  1.0 : Filosofia de vida que busca excluir, sempre que praticável, todas as formas de exploração animal. Tanto na alimentação, quanto no entretenimento, no vestuário e em todos os outros comportamentos de consumo. É também uma postura política e social contra a exploração animal.
Me:O que é leite de coco?
ChatBot  0.91 : Qualquer leite que seja feito a partir de vegetais. Os mais comuns são os de soja, amêndoas, amendoim e coco. Basicamente, é só bater o vegetal com o dobro de água e coar, mas é importante buscar receitas específicas.
Me:quem é fabio chaves?
ChatBot  0.0 : Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do que em outros tipos de abate.
Me:fabio chaves
ChatBot  0.0 : Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do

In [55]:
# Me:oi
# ChatBot  1.0 : oi
# Me:boa tarde
# ChatBot  1.0 : boa tarde
# Me:o que e veganismo?
# ChatBot  1.0 : Filosofia de vida que busca excluir, sempre que praticável, todas as formas de exploração animal. Tanto na alimentação, quanto no entretenimento, no vestuário e em todos os outros comportamentos de consumo. É também uma postura política e social contra a exploração animal.
# Me:O que é leite de coco?
# ChatBot  0.91 : Qualquer leite que seja feito a partir de vegetais. Os mais comuns são os de soja, amêndoas, amendoim e coco. Basicamente, é só bater o vegetal com o dobro de água e coar, mas é importante buscar receitas específicas.
# Me:quem é fabio chaves?
# ChatBot  0.0 : Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do que em outros tipos de abate.
# Me:fabio chaves
# ChatBot  0.0 : Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do que em outros tipos de abate.
# Me:salsicha de soja
# ChatBot  0.88 : Procure por PTS.
# Me:pts
# ChatBot  1.0 : PTS é uma sigla que significa Proteína Texturizada de Soja. É a forma mais comum e fácil de se fazer a "carne de soja", que se parece muito com carne moída. A PTS é obtida da soja por meio de um processo industrial chamado extrusão. Muitos nutricionistas não recomendam a ingestão exagerada de PTS e alguns recomendam que ela não seja consumida.
# Me:falafel
# ChatBot  1.0 : Bolinho frito à base de grão-de-bico e ervas em formato ovalado de origem árabe. Sua receita original é vegana. É simplesmente delicioso.
# Me:b 12
# ChatBot  1.0 : Vitamina muito falada no meio vegano por ser encontrada naturalmente apenas em produtos de origem animal. Ela não é, no entanto, produzida por animais. A vitamina B12 é produzida por bactérias e é encontrada em produtos animais porque eles tornam-se depósitos dessa vitamina por consumir alimentos sem muita higienização. A vitamina B12 é essencial para a prevenção de alguns tipos de anemia e para o bom funcionamento do sistema nervoso. Sua reposição por meio de suplementação é obrigatória para vegetarianos. Muitas pessoas que comem carne também apresentam deficiência dessa vitamina e também devem fazer a suplementação.
# Me:ferro
# ChatBot  0.66 : Melado ou melado-de-cana é um derivado da cana-de-açúcar usado para substituir o mel em receitas. A receita vegana mais famosa que utiliza este ingrediente é o pão de melado. O melado é riquíssimo em ferro, mais uma vantagem sobre o mel, além de ser livre de crueldade.
# Me:ferritina
# ChatBot  0.0 : Um eufemismo para definir o assassinato de animais na pecuária. No abate humanitário, segundo a indústria, os animais sofrem menos do que em outros tipos de abate.
# Me:tchau
# ChatBot  1.0 : tchau
